In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# import the necessary packages
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os

In [0]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = os.path.sep.join(['/content/gdrive/My Drive/app/Face_Recognition_PyImage/face_detection_model', "deploy.prototxt"])
modelPath = os.path.sep.join(['/content/gdrive/My Drive/app/Face_Recognition_PyImage/face_detection_model',
	"res10_300x300_ssd_iter_140000.caffemodel"])
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
 
# load our serialized face embedding model from disk
print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch('/content/gdrive/My Drive/app/Face_Recognition_PyImage/openface_nn4.small2.v1.t7')

[INFO] loading face detector...


error: ignored

In [0]:
# grab the paths to the input images in our dataset
print("[INFO] quantifying faces...")
imagePaths = list(paths.list_images('/content/gdrive/My Drive/app/Face_Recognition_PyImage/datatqm'))

# initialize our lists of extracted facial embeddings and
# corresponding people names
knownEmbeddings = []
knownNames = []

# initialize the total number of faces processed
total = 0

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
	# extract the person name from the image path
	print("[INFO] processing image {}/{}".format(i + 1,
		len(imagePaths)))
	name = imagePath.split(os.path.sep)[-2]

	# load the image, resize it to have a width of 600 pixels (while
	# maintaining the aspect ratio), and then grab the image
	# dimensions
	image = cv2.imread(imagePath)
	image = imutils.resize(image, width=600)
	(h, w) = image.shape[:2]

	# construct a blob from the image
	imageBlob = cv2.dnn.blobFromImage(
		cv2.resize(image, (300, 300)), 1.0, (300, 300),
		(104.0, 177.0, 123.0), swapRB=False, crop=False)

	# apply OpenCV's deep learning-based face detector to localize
	# faces in the input image
	detector.setInput(imageBlob)
	detections = detector.forward()

	# ensure at least one face was found
	if len(detections) > 0:
		# we're making the assumption that each image has only ONE
		# face, so find the bounding box with the largest probability
		i = np.argmax(detections[0, 0, :, 2])
		confidence = detections[0, 0, i, 2]

		# ensure that the detection with the largest probability also
		# means our minimum probability test (thus helping filter out
		# weak detections)
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the face
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# extract the face ROI and grab the ROI dimensions
			face = image[startY:endY, startX:endX]
			(fH, fW) = face.shape[:2]

			# ensure the face width and height are sufficiently large
			if fW < 20 or fH < 20:
				continue

			# construct a blob for the face ROI, then pass the blob
			# through our face embedding model to obtain the 128-d
			# quantification of the face
			faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
				(96, 96), (0, 0, 0), swapRB=True, crop=False)
			embedder.setInput(faceBlob)
			vec = embedder.forward()

			# add the name of the person + corresponding face
			# embedding to their respective lists
			knownNames.append(name)
			knownEmbeddings.append(vec.flatten())
			total += 1

# dump the facial embeddings + names to disk
print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open('/content/gdrive/My Drive/app/Face_Recognition_PyImage/output1/encodings123.pickle', "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] quantifying faces...
[INFO] processing image 1/258
[INFO] processing image 2/258
[INFO] processing image 3/258
[INFO] processing image 4/258
[INFO] processing image 5/258
[INFO] processing image 6/258
[INFO] processing image 7/258
[INFO] processing image 8/258
[INFO] processing image 9/258
[INFO] processing image 10/258
[INFO] processing image 11/258
[INFO] processing image 12/258
[INFO] processing image 13/258
[INFO] processing image 14/258
[INFO] processing image 15/258
[INFO] processing image 16/258
[INFO] processing image 17/258
[INFO] processing image 18/258
[INFO] processing image 19/258
[INFO] processing image 20/258
[INFO] processing image 21/258
[INFO] processing image 22/258
[INFO] processing image 23/258
[INFO] processing image 24/258
[INFO] processing image 25/258
[INFO] processing image 26/258
[INFO] processing image 27/258
[INFO] processing image 28/258
[INFO] processing image 29/258
[INFO] processing image 30/258
[INFO] processing image 31/258
[INFO] processing im

Train face recognition model

In [0]:
# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import argparse
import pickle

In [0]:
# load the face embeddings
print("[INFO] loading face embeddings...")
data = pickle.loads(open('/content/gdrive/My Drive/app/Face_Recognition_PyImage/output1/embeddings.pickle', "rb").read())
#with open('/content/gdrive/My Drive/app/Face_Recognition_PyImage/output1/embeddings.pickle', 'rb') as my_dump_file:
 #   data = pickle.load(my_dump_file)
  #  data = np.array(data)

# encode the labels
print("[INFO] encoding labels...")
le = LabelEncoder()
labels = le.fit_transform(data["names"])
# train the model used to accept the 128-d embeddings of the face and
# then produce the actual face recognition
print("[INFO] training model...")
recognizer = SVC(C=1.0, kernel="linear", probability=True)
recognizer.fit(data["embeddings"], labels)


# write the actual face recognition model to disk
f = open('/content/gdrive/My Drive/app/Face_Recognition_PyImage/output1/recognizer.pickle', "wb")
f.write(pickle.dumps(recognizer))
f.close()
 
# write the label encoder to disk
f = open('/content/gdrive/My Drive/app/Face_Recognition_PyImage/output1/le.pickle', "wb")
f.write(pickle.dumps(le))
f.close()
print("finish!")

[INFO] loading face embeddings...
[INFO] encoding labels...
[INFO] training model...
finish!
